# Big Blue Bus Route Analysis

In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql, magics
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
import utils

In [3]:
import altair as alt
from dla_utils import _dla_utils

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

In [4]:
bbb = 300

In [5]:
rt_sched = utils.read_data()

In [6]:
#filtering data down to one operator for a route analysis
rt_sched = rt_sched>>filter(_.calitp_itp_id == bbb)

In [7]:
rt_sched.sample(5)

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month
997,300,Big Blue Bus,0,3487,9,2022-05-31,2022-03-08,2022-09-01,68,50,0.74,Tuesday,May
1189,300,Big Blue Bus,0,3493,17,2022-05-24,2022-03-08,2022-09-01,88,88,1.00,Tuesday,May
1307,300,Big Blue Bus,0,3495,R3,2022-05-18,2022-03-08,2022-09-01,51,51,1.00,Wednesday,May
6618,300,Big Blue Bus,0,3483,5,2022-05-13,2022-03-08,2022-09-01,24,24,1.00,Friday,May
1373,300,Big Blue Bus,0,3496,R7,2022-05-19,2022-03-08,2022-09-01,76,76,1.00,Thursday,May


In [8]:
rt_sched_agg = utils.agg_by_date(rt_sched,'num_sched', 'num_vp')

In [9]:
rt_sched_agg

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
0,300,Big Blue Bus,0,2022-05-01,Sunday,May,770,526,0.68
1,300,Big Blue Bus,0,2022-05-02,Monday,May,1516,1066,0.70
2,300,Big Blue Bus,0,2022-05-03,Tuesday,May,1516,1365,0.90
3,300,Big Blue Bus,0,2022-05-04,Wednesday,May,1516,1381,0.91
4,300,Big Blue Bus,0,2022-05-05,Thursday,May,1516,1343,0.89
...,...,...,...,...,...,...,...,...,...
56,300,Big Blue Bus,0,2022-06-26,Sunday,June,770,498,0.65
57,300,Big Blue Bus,0,2022-06-27,Monday,June,1461,1013,0.69
58,300,Big Blue Bus,0,2022-06-28,Tuesday,June,1461,1359,0.93
59,300,Big Blue Bus,0,2022-06-29,Wednesday,June,1461,1367,0.94


In [10]:
## Looking at Routes

In [11]:
rt_sched>>count(_.route_id)>>arrange(-_.n)

,route_id,n
0,3479,61
1,3480,61
2,3481,61
4,3485,61
5,3486,61
6,3487,61
8,3489,61
9,3490,61
12,3493,61
13,3494,61


In [12]:
rt_sched>>filter(_.service_date=='2022-06-01')>>summarize(n_routes = _.route_id.nunique())

,n_routes
0,19


In [13]:
(rt_sched>>group_by(_.service_date)>>summarize(n_routes=_.route_id.nunique())>>arrange(-_.n_routes)).n_routes.describe()

count   61.00
mean    16.43
std      3.92
min     10.00
25%     11.00
50%     19.00
75%     19.00
max     19.00
Name: n_routes, dtype: float64

In [14]:
#checking one route
(rt_sched>>filter(_.route_id=='3488')
).head()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month
5723,300,Big Blue Bus,0,3488,R10,2022-06-06,2022-03-08,2022-09-01,6,4,0.67,Monday,June
5724,300,Big Blue Bus,0,3488,R10,2022-05-02,2022-03-08,2022-09-01,6,4,0.67,Monday,May
5725,300,Big Blue Bus,0,3488,R10,2022-05-16,2022-03-08,2022-09-01,6,4,0.67,Monday,May
5726,300,Big Blue Bus,0,3488,R10,2022-06-13,2022-03-08,2022-09-01,6,4,0.67,Monday,June
5727,300,Big Blue Bus,0,3488,R10,2022-05-31,2022-03-08,2022-09-01,6,4,0.67,Tuesday,May


In [47]:
rt_sched.service_date.info()

<class 'pandas.core.series.Series'>
Int64Index: 1002 entries, 681 to 6897
Series name: service_date
Non-Null Count  Dtype         
--------------  -----         
1002 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 15.7 KB


In [50]:
highlight = alt.selection(type='single', on="mouseover", 
                              fields=['route_short_name', "pct_w_vp"], nearest=True)
selection = alt.selection_multi(
                          fields=['route_short_name'], bind='legend')

line = alt.Chart(rt_sched).mark_line().encode(
    x=alt.X('service_date:T', title=_dla_utils.labeling('service_date')),
    y=alt.Y('pct_w_vp', title=('Percent with Vehicle Positions')),
    color='route_short_name',
    strokeDash='route_short_name',
      tooltip = alt.Tooltip(["route_short_name", "service_date","pct_w_vp"]),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
                size=alt.condition(~highlight, alt.value(2), alt.value(5))
             ).properties(title={
                 "text": ["Big Blue Bus: Average Percent of Scheduled Trips with Vehicle Postions Data", "by Route"],
             }).add_selection(
                 selection, highlight).properties(width=600)
# line_chart = styleguide.preset_chart_config(line)
# line_chart = _dla_utils.add_tooltip(line_chart, 'route_short_name', 'pct_w_vp')
# line_chart.properties(width=600)
line

TypeError: Object of type date is not JSON serializable

alt.Chart(...)

In [32]:
rt_sched>>filter(_.pct_w_vp<.5)>>count(_.route_id)

,route_id,n
0,3489,1
1,3491,1
2,3501,40


* something funky happening with route 3501. 
* checking trip updates

In [33]:
(rt_sched>>filter(_.route_id=='3501')>>arrange(_.service_date))

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month
1550,300,Big Blue Bus,0,3501,44,2022-05-02,2022-03-08,2022-09-01,104,0,0.00,Monday,May
1556,300,Big Blue Bus,0,3501,44,2022-05-03,2022-03-08,2022-09-01,104,0,0.00,Tuesday,May
1565,300,Big Blue Bus,0,3501,44,2022-05-04,2022-03-08,2022-09-01,104,0,0.00,Wednesday,May
1575,300,Big Blue Bus,0,3501,44,2022-05-05,2022-03-08,2022-09-01,104,0,0.00,Thursday,May
1553,300,Big Blue Bus,0,3501,44,2022-05-06,2022-03-08,2022-09-01,52,0,0.00,Friday,May
1574,300,Big Blue Bus,0,3501,44,2022-05-09,2022-03-08,2022-09-01,104,0,0.00,Monday,May
1559,300,Big Blue Bus,0,3501,44,2022-05-10,2022-03-08,2022-09-01,104,0,0.00,Tuesday,May
1548,300,Big Blue Bus,0,3501,44,2022-05-11,2022-03-08,2022-09-01,104,0,0.00,Wednesday,May
1568,300,Big Blue Bus,0,3501,44,2022-05-12,2022-03-08,2022-09-01,104,0,0.00,Thursday,May
1560,300,Big Blue Bus,0,3501,44,2022-05-13,2022-03-08,2022-09-01,52,0,0.00,Friday,May


In [34]:

## seeing what the pct_with_rt would look like without route 3501

In [35]:
no_3501 = utils.agg_by_date((rt_sched>>filter(_.route_id != '3501')),'num_sched', 'num_vp')

In [36]:
no_3501.sample()

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
24,300,Big Blue Bus,0,2022-05-25,Wednesday,May,1412,1348,0.95


In [37]:
with_3501 = utils.agg_by_date((rt_sched),'num_sched', 'num_vp')
with_3501.sample()

,calitp_itp_id,agency_name,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_vp,pct_w_vp
57,300,Big Blue Bus,0,2022-06-27,Monday,June,1461,1013,0.69


In [38]:

line_1 = (alt.Chart(with_3501).mark_line(color='navy').encode(
    x=alt.X('service_date', title=_dla_utils.labeling('service_date')),
    y=alt.Y('pct_w_vp', title=_dla_utils.labeling('Percent with Vehicle Positions')),
    opacity = alt.Opacity('with_3501:Q', legend=alt.Legend(title="With 3501"))
))
line_2 = alt.Chart(no_3501).mark_line(color='orange').encode(
    x=alt.X('service_date', title=_dla_utils.labeling('service_date')),
    y=alt.Y('pct_w_vp', title=_dla_utils.labeling('Percent with Vehicle Positions')),
    opacity= alt.Opacity('no_3501:Q', legend=alt.Legend(title="Without 3501")))
chart = line_1+line_2
chart
# line_chart = styleguide.preset_chart_config(chart)
# line_chart = _dla_utils.add_tooltip(line_chart, 'service_date:O', 'pct_w_vp:Q')
# line_chart.properties(width=600)


alt.LayerChart(...)

In [39]:
rt_sched.sample()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month
5932,300,Big Blue Bus,0,3488,R10,2022-06-15,2022-03-08,2022-09-01,6,6,1.00,Wednesday,June


In [40]:
utils.groupby_onecol((rt_sched), 'month', 'pct_w_vp')

,month,avg
1,May,0.84
0,June,0.86


In [41]:
rt_sched.sample()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month
1113,300,Big Blue Bus,0,3490,14,2022-05-11,2022-03-08,2022-09-01,107,103,0.96,Wednesday,May


In [42]:
route_weekday = (utils.groupby_twocol((rt_sched), 'route_short_name', 'weekday', 'pct_w_vp', 'weekday'))

In [43]:
route_weekday.sample()

,route_short_name,weekday,avg
91,9,Thursday,0.97


In [44]:
cats_day = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']


In [45]:
highlight = alt.selection(type='single', on="mouseover", 
                              fields=['route_short_name', "avg"], nearest=True)
selection = alt.selection_multi(
                          fields=['route_short_name'], bind='legend')

line = alt.Chart(route_weekday).mark_line(point={
      "filled": False,
      "fill": "white"
    }).encode(
    x=alt.X('weekday', title=_dla_utils.labeling('weekday'),  sort=cats_day),
    y=alt.Y('avg:Q', title=('Average Percent with Vehicle Positions')),
    color='route_short_name',
  #  strokeDash='route_short_name',
    tooltip = alt.Tooltip(["route_short_name", "weekday","avg"]),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
                size=alt.condition(~highlight, alt.value(2), alt.value(5))
             ).properties(title={
                 "text": ["BBB Average Percent of Scheduled Trips with Vehicle Postions Data", "by Route"],
             }).add_selection(
                 selection, highlight).properties(width=600)
line
#line_chart = styleguide.preset_chart_config(line)

alt.Chart(...)